In [12]:
!kaggle datasets files "ashirwadsangwan/imdb-dataset"
!kaggle datasets download "ashirwadsangwan/imdb-dataset" --unzip -f "title.ratings.tsv.gz" 

name                      size  creationDate         
-----------------------  -----  -------------------  
title.principals.tsv.gz  290MB  2019-11-18 18:36:47  
title.akas.tsv.gz        158MB  2019-11-18 18:36:47  
title.ratings.tsv.gz       5MB  2019-11-18 18:36:47  
name.basics.tsv.gz       180MB  2019-11-18 18:36:47  
title.basics.tsv.gz      108MB  2019-11-18 18:36:47  


usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: unrecognized arguments: title.ratings.tsv.gz


In [22]:
import csv
import json
def createBaskets(sourceFile, destFile):
    tsv = csv.reader(open(sourceFile, encoding="utf-8"), delimiter="\t")
    c=0
    tuples = {}
    for row in tsv:
        if row[3]!="actor" and row[3]!="actress":
            continue
        if c == 0:
            c = c+1
            continue
        if row[0] in tuples.keys():
            tuples[row[0]] = tuples[row[0]] + [row[2]]
        else:
            tuples[row[0]] = [row[2]]

        if c==101:
            break;
        c = c+1
    json.dump(tuples, open(destFile+'.json', 'w'))

In [23]:
createBaskets("data.tsv", "preprocessed")

In [30]:
loaded = json.load(open("preprocessed.json", encoding="utf-8"))

{'tt0000001': ['nm0005690', 'nm0374658'],
 'tt0000002': ['nm0721526', 'nm1335271'],
 'tt0000003': ['nm0721526', 'nm5442194', 'nm1335271', 'nm5442200'],
 'tt0000004': ['nm0721526', 'nm1335271'],
 'tt0000005': ['nm0443482', 'nm0653042', 'nm0005690', 'nm0249379'],
 'tt0000006': ['nm0005690'],
 'tt0000007': ['nm0179163',
  'nm0183947',
  'nm0005690',
  'nm0374658',
  'nm0249379'],
 'tt0000008': ['nm0653028', 'nm0005690', 'nm0374658'],
 'tt0000009': ['nm0063086', 'nm0183823', 'nm1309758', 'nm0085156'],
 'tt0000010': ['nm0525910'],
 'tt0000011': ['nm3692297', 'nm0804434'],
 'tt0000012': ['nm2880396',
  'nm9735580',
  'nm0525900',
  'nm9735581',
  'nm0525908',
  'nm0525910',
  'nm9735579',
  'nm9653419'],
 'tt0000013': ['nm0525908', 'nm1715062', 'nm0525910'],
 'tt0000014': ['nm0166380', 'nm0244989', 'nm0525910'],
 'tt0000015': ['nm0721526'],
 'tt0000016': ['nm0525900', 'nm9735581', 'nm0525910'],
 'tt0000017': ['nm3691272', 'nm3692829', 'nm1587194', 'nm0804434'],
 'tt0000018': ['nm3692071', 'n

In [24]:
tsv = csv.reader(open("name.tsv", encoding="utf-8"), delimiter="\t")
for row in tsv:
    if row[0] == "nm0617588":
        print(row)

['nm0617588', 'Georges Méliès', '1861', '1938', 'director,actor,producer', 'tt0000499,tt0002113,tt0000091,tt0223267']
